In [1]:
import pandas as pd
import numpy as np

In [2]:
dfs = []
for i in range(19):
    dfs.append(pd.read_parquet(f"/net/scratch2/muchane/new_tokens/{i}.parquet"))

df = pd.concat(dfs)

In [3]:
df["seq_len"] = df["tokens"].apply(len)

In [6]:
df[df["lang_id"]==0]

,uid,article_id,lang_id,tokens,seq_len
256,41726489,116715,0,"[128000, 54, 408, 2176, 12336, 320, 16381, 755...",111
469,44292755,66915130,0,"[128000, 98982, 1697, 28845, 369, 81745, 42509...",563
470,44292756,66915132,0,"[128000, 67291, 360, 2259, 9235, 300, 11, 2663...",69
471,44292757,66915138,0,"[128000, 43346, 26888, 374, 264, 2363, 555, 27...",646
472,44292767,53839293,0,"[128000, 5953, 67, 360, 90251, 21252, 462, 314...",143
...,...,...,...,...,...
16766881,53913744,33579634,0,"[128000, 53, 648, 11906, 34569, 374, 264, 1514...",70
16766882,53913745,33579640,0,"[128000, 45, 438, 67455, 44957, 21016, 352, 13...",236
16766883,53913746,33579641,0,"[128000, 50135, 34569, 374, 264, 15140, 315, 2...",119
16766884,53913747,33579650,0,"[128000, 3976, 42861, 374, 279, 836, 315, 459,...",7087


In [23]:
s = df[df["lang_id"]==0]["seq_len"]
intervals = [(16, 64), (64, 256), (256, 1024), (1024, 4096), (4096, np.inf)]

# Count the number of values in each interval using list comprehension
counts = [((s >= low)).sum() for (low, high) in intervals]

In [24]:
counts

[6647703, 6092657, 3775081, 1143319, 138512]

In [26]:
x = np.array([100E6, 200E6, 400E6, 200E6, 100E6])//np.array([32, 128, 512, 2048, 8192])

In [33]:
x

array([3125000., 1562500.,  781250.,   97656.,   12207.])

In [29]:
np.cumsum(x[::-1])[::-1] 

array([5578613., 2453613.,  891113.,  109863.,   12207.])

In [ ]:
200E6/8192

In [5]:
df["bin"] = None

In [6]:
for i, n in [(8192, 100E6), (2048, 200E6), (512, 400E6), (128, 200E6), (32, 100E6)]:
    seq_per_context = 2**17//i
    raw_count = (n//i)
    need = int(raw_count - (raw_count%seq_per_context) + seq_per_context)
    for_bin = df[(df["lang_id"] == 0) & (df["bin"].isnull()) & (df["seq_len"] >= i)].sample(need)
    df.loc[for_bin.index, "bin"] = i

In [53]:
for_bin = df[(df["lang_id"] == 0) & (df["bin"].isnull()) & (df["seq_len"] >= 8192)].sample(12208)
    #for_bin["bin"] = i

In [59]:
df.loc[for_bin.index, "bin"] = 8192

In [70]:
df

,uid,article_id,lang_id,tokens,seq_len,bin
0,21196565,10381928,1,"[128000, 48889, 261, 51301, 73302, 3903, 6754,...",1665,None
1,21196584,7981279,1,"[128000, 38, 301, 754, 4171, 69, 839, 4289, 23...",819,None
2,21196585,7981283,1,"[128000, 7, 3753, 18, 8, 426, 564, 320, 4468, ...",138,None
3,21196586,7981286,1,"[128000, 1542, 604, 350, 4843, 268, 65, 11252,...",984,None
4,21196587,7981290,1,"[128000, 18674, 15832, 309, 2357, 68, 17360, 3...",495,None
...,...,...,...,...,...,...
17169402,1949591,189024,30,"[128000, 5619, 239, 100335, 102650, 100907, 10...",6064,None
17169403,1949592,189025,30,"[128000, 109159, 100361, 105909, 120977, 10036...",1911,None
17169404,1949593,189030,30,"[128000, 80338, 86133, 102672, 122033, 24810, ...",233,None
17169405,1949594,189035,30,"[128000, 88344, 100537, 100361, 100395, 100348...",85,None


In [7]:
seq_per_context = 2**17//256
raw_count = (1E9//256)
need = int(raw_count - (raw_count%seq_per_context) + seq_per_context)
for_bin = df[(df["lang_id"] != 0) & (df["bin"].isnull()) & (df["seq_len"] >= 256)].sample(need)
df.loc[for_bin.index, "bin"] = 256

In [74]:
df.loc[df["lang_id"] != 0, "bin"] = None

In [80]:
df[~df["bin"].isnull()]["lang_id"].value_counts()

lang_id
0     5579056
1     1257191
2      946444
3      706842
7      570780
6      316596
23      69016
30      39691
Name: count, dtype: Int64

In [8]:
df = df.sort_values(["lang_id", "bin"])

In [9]:
def concat_seq(row):
    if row["bin"]:
        ret = row["tokens"][:row["bin"]]
        ret[-1] = 128009
        return ret

In [10]:
new_col = df.apply(concat_seq, axis=1)

In [11]:
df["to_pack"] = new_col

In [12]:
df.to_parquet("/net/projects/interp/new_tokens.parquet")

In [13]:
from transformers import PreTrainedTokenizerFast

/net/scratch/muchane/.pyenv/versions/3.11.9/envs/nnmf-jax/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [14]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("meta-llama/Llama-3.2-3B",add_eos_token=True)

In [15]:
tokenizer.decode(df.iloc[0]["to_pack"].tolist())

2024-10-14 10:49:34.163134: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-14 10:49:39.626821: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-14 10:49:51.089175: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


'<|begin_of_text|>Wendee Lee (born February 20, 1960 in Los Angeles, California) is an American voice actress. She is best known for<|eot_id|>'

In [1]:
import pandas as pd
import tensorstore as ts
import numpy as np

In [2]:
df = pd.read_parquet("/net/projects/interp/new_tokens.parquet")

In [3]:
to_save = df[~df["bin"].isnull()]

In [6]:
test = to_save[(to_save["lang_id"] == 0) & (to_save["bin"] == 8192)]

In [9]:
x = np.array(test["to_pack"].tolist())

In [11]:
y = x.reshape(-1,2**17)

In [18]:
to_write_eng = []
others = []
for name, group in to_save.groupby(["lang_id", "bin"]):
    if name[0] == 0:
        to_write.append((name, np.array(group["to_pack"].tolist()).reshape(-1,2**17)))
    else:
        others.append((name, np.array(group["to_pack"].tolist())))

In [19]:
x = np.concatenate([a[1] for a in others])

In [22]:
x = x.reshape(-1,2**17)

In [27]:
x_0 = np.concatenate([a[1] for a in to_write[:5]])

In [29]:
final_to_write = np.concatenate([x_0,x])

In [30]:
final_to_write.shape

(15260, 131072)

In [31]:
    tokens_data = ts.open(
        {
        'driver': 'zarr3',
        'cache_pool': {'total_bytes_limit': 1E9},
        'recheck_cached_data': 'open',
        'kvstore': {
            'driver': 'file',
            'file_io_concurrency': {'limit': 2048},
            'path': '/net/scratch2/muchane/tokens',
            },
        'create': True,
        },
        dtype=ts.int64,
        chunk_layout=ts.ChunkLayout(
        write_chunk_shape=[763, 8192],
        ),
        shape=[15260, 131072],
    ).result()

In [32]:
tokens_data.write(final_to_write).result()

In [39]:
test_loc = tokens_data[15260-1].read().result()

In [40]:
test_loc.reshape(-1,256)[-1]

array([128000, 109159, 100361, 105909, 120977, 100364, 103568, 100907,
       100293, 101877, 102399, 103477, 107098, 100361, 105909, 120977,
       100276, 100400,  65804,  48909,  44747, 100549,  84736, 100799,
       103156,  69258, 100460, 100497, 100428, 100460, 100287, 100391,
        44747, 103887, 109953,  48909, 108500,  44747,  85410, 100436,
       107098, 100361, 105909, 120977, 100364, 103568, 100907, 100293,
       101877, 102399, 103477,    320, 109159, 109159, 100714,  92911,
            8,  92317, 100273, 101483,    220,   1049,     15,  92317,
       100271, 107098, 100366,  44747,  48909, 100580, 101166,  86133,
        69258,  35470, 102411, 117229, 101495, 105528, 102648, 100311,
        48909,  24810, 100291, 106269,  61196,  85410, 101166,  73753,
        69258, 100750, 101247, 100428,  48909, 100322,  24810, 101159,
       105023, 109159, 100361, 105909, 120977, 100364, 103568, 100907,
       100276, 100400,  65804, 100929, 100280,  48909, 100273, 109553,
      

In [34]:
to_save.re

,uid,article_id,lang_id,tokens,seq_len,bin,to_pack
256,41726489,116715,0,"[128000, 54, 408, 2176, 12336, 320, 16381, 755...",111,32.0,"[128000, 54, 408, 2176, 12336, 320, 16381, 755..."
470,44292756,66915132,0,"[128000, 67291, 360, 2259, 9235, 300, 11, 2663...",69,32.0,"[128000, 67291, 360, 2259, 9235, 300, 11, 2663..."
512,44934285,689146,0,"[128000, 57735, 640, 546, 72, 320, 51, 89, 640...",96,32.0,"[128000, 57735, 640, 546, 72, 320, 51, 89, 640..."
515,44934288,689149,0,"[128000, 20027, 5724, 320, 112638, 81, 5724, 8...",77,32.0,"[128000, 20027, 5724, 320, 112638, 81, 5724, 8..."
516,44934289,689150,0,"[128000, 78054, 269, 5641, 84, 320, 40, 114579...",81,32.0,"[128000, 78054, 269, 5641, 84, 320, 40, 114579..."
...,...,...,...,...,...,...,...
17168985,1946235,1282946,30,"[128000, 45279, 117765, 92911, 69258, 100431, ...",384,256.0,"[128000, 45279, 117765, 92911, 69258, 100431, ..."
17169002,1946252,1283015,30,"[128000, 100574, 100305, 100497, 100282, 10102...",553,256.0,"[128000, 100574, 100305, 100497, 100282, 10102..."
17169298,1947987,1283230,30,"[128000, 80338, 95048, 79468, 103039, 100296, ...",1758,256.0,"[128000, 80338, 95048, 79468, 103039, 100296, ..."
17169402,1949591,189024,30,"[128000, 5619, 239, 100335, 102650, 100907, 10...",6064,256.0,"[128000, 5619, 239, 100335, 102650, 100907, 10..."


In [44]:
x_0_desc = np.concatenate([np.repeat(a[0][1], a[1].shape[0]) for a in to_write[:5]])

In [45]:
x_0_desc.shape

(7630,)

In [46]:
x_desc = np.repeat(256,x.shape[0])

In [49]:
x_desc_towrite = np.concatenate([x_0_desc,x_desc])

In [52]:
x_desc_towrite = x_desc_towrite.reshape((15260,1))

In [53]:
x_desc_towrite.shape

(15260, 1)

In [54]:
    packing_data = ts.open(
        {
        'driver': 'zarr3',
        'cache_pool': {'total_bytes_limit': 1E9},
        'recheck_cached_data': 'open',
        'kvstore': {
            'driver': 'file',
            'file_io_concurrency': {'limit': 2048},
            'path': '/net/scratch2/muchane/packing',
            },
        'create': True,
        },
        dtype=ts.int64,
        chunk_layout=ts.ChunkLayout(
        write_chunk_shape=[15260, 1],
        ),
        shape=[15260, 1],
    ).result()

In [56]:
packing_data.write(x_desc_towrite.astype(int)).result()